In [6]:
dataset = "dataset"

In [2]:
%pip install librosa

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 932.9 kB/s eta 0:00:03
   ------- -------------------------------- 0.5/2.7 MB 932.9 kB/s eta 0:00:03
   ----------- ---------------------------- 0.8/2.7 MB 780.2 kB/s eta 0:00:03
   --------------- ------------------------ 1.0/2.7 MB 882.6 kB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 883.4 kB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 883.4 kB/s eta 0:00:02
   ---------------------- ----------------- 1.6/2.7 MB 864.6 kB/s eta 0:00:02
   -------------------------- ------------- 1.8/2.7 MB 883.1 kB/s eta 0:00:02
   ------------------------------ 

In [3]:


import os
import librosa
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
DATASET_PATH = dataset
SAMPLE_RATE = 16000
N_MFCC = 40

X = []
y = []

In [8]:
# -------- LOAD DATA --------
for label in os.listdir(DATASET_PATH):
    label_path = os.path.join(DATASET_PATH, label)
    if not os.path.isdir(label_path):
        continue

    for file in os.listdir(label_path):
        if not file.lower().endswith(".wav"):
            continue

        file_path = os.path.join(label_path, file)
        audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=N_MFCC)
        mfcc_mean = np.mean(mfcc.T, axis=0)

        X.append(mfcc_mean)
        y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset shape:", X.shape, y.shape)

Dataset shape: (48, 40) (48,)


In [9]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# -------- TRAIN / TEST SPLIT --------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

In [10]:
# -------- SCALING (EXPLICIT) --------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------- MODEL (RAW) --------
model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)
model.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [11]:
y_pred = model.predict(X_test_scaled)

In [12]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

       click       1.00      1.00      1.00         3
 open_chrome       0.75      1.00      0.86         3
open_notepad       1.00      0.50      0.67         2
 scroll_down       1.00      1.00      1.00         2
   scroll_up       1.00      1.00      1.00         2

    accuracy                           0.92        12
   macro avg       0.95      0.90      0.90        12
weighted avg       0.94      0.92      0.91        12


Confusion Matrix:
[[3 0 0 0 0]
 [0 3 0 0 0]
 [0 1 1 0 0]
 [0 0 0 2 0]
 [0 0 0 0 2]]


In [13]:
import pickle

model_bundle = {
    "model": model,
    "scaler": scaler,
    "label_encoder": label_encoder
}

with open("voice_command_raw_model.pkl", "wb") as f:
    pickle.dump(model_bundle, f)

print("✅ Pickle file created: voice_command_raw_model.pkl")


✅ Pickle file created: voice_command_raw_model.pkl


In [ ]:
import pickle

with open("voice_command_raw_model.pkl", "rb") as f:
    data = pickle.load(f)

model = data["model"]
scaler = data["scaler"]
label_encoder = data["label_encoder"]

print("Loaded classes:", label_encoder.classes_)
